In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
import matlab.engine
import socket, struct
import threading
import concurrent.futures
import random
from tensorflow.keras.layers import Dense, Input
import tensorflow as tf

# Check if GPU is available
if tf.config.list_physical_devices('GPU'):
    print("GPU is available")
else:
    print("GPU is not available")


2024-04-24 16:23:21.985047: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-24 16:23:22.469356: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


GPU is not available


2024-04-24 16:23:23.972773: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-24 16:23:23.976654: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [38]:
## matlab api connection
eng = matlab.engine.start_matlab()
eng.cd(r'/home/pvm8318/Documents/Reinforcement/2023b')
eng.addpath(r'/home/pvm8318/Documents/Reinforcement/2023b')
def SimRun():
    eng.sim('Buck_Converter.slx')
    return

SyntaxError: invalid syntax (3264257124.py, line 1)

In [1]:
import tensorflow as tf

print("TensorFlow version: ", tf.__version__)
print("Is GPU available: ", tf.test.is_gpu_available())
print("CUDA enabled: ", tf.test.is_built_with_cuda())

2024-04-24 16:55:38.660879: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-24 16:55:39.144937: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


TensorFlow version:  2.16.1
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
Is GPU available:  False
CUDA enabled:  True


2024-04-24 16:55:39.466802: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-24 16:55:39.470605: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [1]:
import torch
print(torch.version.cuda)

12.1


In [39]:
## TCP Connection
MESSAGE_SIZE = 24
DELIMITER = b'\n'
TCP_IP = '127.0.0.1'
TCP_PORT = 50000
BUFFER_SIZE = MESSAGE_SIZE if MESSAGE_SIZE else 32  # Minimum for two doubles


def send_data(conn, val):
    """Sends two double-precision numbers."""
    # Fixed Size
    msg = struct.pack('>d', val)
    conn.send(msg)

def receive_data(conn):
    """Receives three double-precision numbers."""
    if MESSAGE_SIZE:
        data = conn.recv(MESSAGE_SIZE)
        val1, val2, Time = struct.unpack('>ddd', data)
    else:
        # Delimiter
        val1 = None
        val2 = None
        Time = None
        while True:
            data = conn.recv(BUFFER_SIZE)
            if DELIMITER in data:
                val1_bytes, remaining = data.split(DELIMITER, 1)
                val1 = struct.unpack('>d', val1_bytes)[0]
                if DELIMITER in remaining:
                    val2_bytes, time_bytes = remaining.split(DELIMITER, 1)
                    val2 = struct.unpack('>d', val2_bytes)[0]
                    Time = struct.unpack('>d', time_bytes)[0]
                    break
    return val1, val2, Time

# Close the existing socket connection if it is open

In [40]:
## Buck converter parameters 
Vref = 5
u = 0
R = 1.0  # Resistance
L = 0.1  # Inductance
C = 1e-3  # Capacitance
Vin = 12.0  # Input voltage
Vref = 5.0  # Reference output voltage.0
# State-space representation of the buck converter
A = np.array([[0, 1 / C], [-1 / L, -R / L]])
B = np.array([[0], [1 / L]])
#steady state calculation
duty_cycle =Vref/Vin
Iout = Vref/R
ILref = Iout/duty_cycle

In [41]:
def websocket ():
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.bind((TCP_IP, TCP_PORT))
    print('Waiting for Simulink to start')
    s.listen(1)
    conn, addr = s.accept()
    return conn

In [42]:
def rewardcal(x, u):
    V = x[0]
    IL = x[1]
    Q = 10*np.eye(2)  # State penalty matrix
    R = 1 
    reward = -np.linalg.norm(x - np.array([Vref, ILref]))**2 
    # reward = -np.linalg.norm(x - np.array([Vref, ILref]))**2 - u**2 * R
    return reward


def isdone(x, t):
    # Define the desirable band
    desirable_band = [4.8, 5.2]

    # Initialize the start time and t0
    t0 = None

    V = x[0]
    IL = x[1]
    
    # Check if the state is within the desirable band
    if V >= desirable_band[0] and V <= desirable_band[1]:
        # Check if t0 is None (first time in the band)
        if t0 is None:
            t0 = t
        # Check if the state has been within the desirable band for 0.5 seconds
        elif t - t0 >= 0.5:
            return True
    else:
        # Reset t0 if V gets out of the band
        t0 = None
    
    return False


In [43]:
# close the connection if it is open
try:
    conn.close()
except:
    pass

# Define the shared network architecture
def shared_network(state_dim, action_dim):
    inputs = tf.keras.layers.Input(shape=(state_dim,))
    hidden1 = tf.keras.layers.Dense(64, activation='relu')(inputs)
    hidden2 = tf.keras.layers.Dense(64, activation='relu')(hidden1)
    actor_output = tf.keras.layers.Dense(action_dim, activation='sigmoid')(hidden2)  # Output range: [0, 1]
    critic_output = tf.keras.layers.Dense(1)(hidden2)  # Linear activation for value estimation
    model = tf.keras.Model(inputs=inputs, outputs=[actor_output, critic_output])
    return model

# Example usage
state_dim = 2  # Number of state features (Vc and IL)
action_dim = 1  # Single action (duty cycle)
Vinit = 0
Iinit = 0
max_steps = 30 / 1e-5

# Create the shared network
shared_net = shared_network(state_dim, action_dim)

# Define optimizer and loss functions
actor_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
critic_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

# Actor loss function (policy gradient with advantage)
def actor_loss(log_probs, advantages):
    return -tf.reduce_mean(log_probs * advantages)

# Critic loss function (mean squared error)
def critic_loss(targets, predicted_values):
    return tf.reduce_mean(tf.square(targets - predicted_values))

# Compute advantages for a specific state-action pair
def compute_advantage(q_value, v_value):
    return q_value - v_value

# Training loop
num_episodes = 1000  # Number of episodes
runtime = 30  # Maximum steps per episode
gamma = 0.9  # Discount factor

for episode in range(num_episodes):
    t1 = threading.Thread(target=SimRun)
    t1.start()
    with concurrent.futures.ThreadPoolExecutor() as executor:
        future2 = executor.submit(websocket)
        conn = future2.result()

    # Reset the environment and get the initial state
    state = np.array([Vinit, Iinit])  # Replace with actual initial state
    total_reward = 0
    time = 0
    u = 0

    while time < runtime:
        action_probabilities = shared_net(np.expand_dims(state, axis=0))[0][0]
        action_probs_softmax = tf.nn.softmax(action_probabilities).numpy()
        u = np.random.choice(np.arange(action_dim), p=action_probs_softmax)

        send_data(conn, u)
        val1, val2, time = receive_data(conn)
        next_state = np.array([val1, val2])
        reward = rewardcal(next_state, u)
        done = isdone(next_state, time)

        # Compute the advantage for this state-action pair
        q_value = shared_net(np.expand_dims(state, axis=0))[1][0]
        v_value = shared_net(np.expand_dims(state, axis=0))[1][0]
        advantage = compute_advantage(q_value, v_value)

        # Update the Actor network
        with tf.GradientTape() as tape:
            log_prob = tf.math.log(action_probabilities[u])
            actor_loss_value = actor_loss(log_prob, advantage)
        actor_gradients = tape.gradient(actor_loss_value, shared_net.trainable_variables)
        actor_optimizer.apply_gradients(zip(actor_gradients, shared_net.trainable_variables))

        # Update the Critic network
        with tf.GradientTape() as tape:
            target_value = total_reward
            critic_loss_value = critic_loss(target_value, shared_net(np.expand_dims(state, axis=0))[1][0])
        critic_gradients = tape.gradient(critic_loss_value, shared_net.trainable_variables)
        critic_optimizer.apply_gradients(zip(critic_gradients, shared_net.trainable_variables))

        # Update state for the next step
        state = next_state
        total_reward += reward

        # Print the total reward for the episode
        print("reward is:", reward)
        print('Duty cycle is:', u)
        print('time is:', time)

        if done:
            break  # End the episode if termination condition is met

    # Close the connection
    conn.close()

    print(f"Episode {episode + 1}: Total Reward = {total_reward:.2f}")


Waiting for Simulink to start


ValueError: No gradients provided for any variable.

MATLAB System block 'Buck_Converter/TCP//IP Receive' error occurred when invoking 'stepImpl' method of 'instrument.system.TCPIPReceive'. The error was thrown from '
 <a href="matlab:opentoline('/usr/local/MATLAB/R2023b/toolbox/instrument/instrumentblks/+instrument/+system/TCPIPReceive.m',388, 0)">'/usr/local/MATLAB/R2023b/toolbox/instrument/instrumentblks/+instrument/+system/TCPIPReceive.m'</a> at line 388'.
Caused by:
    Error receiving data from the remote server.
    Additional Information: Operation timed out before requested data was received.

Exception in thread Thread-24 (SimRun):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/home/pvm8318/.local/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_444535/1695